<a href="https://colab.research.google.com/github/kenwkliu/ideas/blob/master/colab/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RAG (Retrieval Augmented Generation)

Based on Shaw Talebi article: https://towardsdatascience.com/how-to-improve-llms-with-rag-abdc132f76ac <br>


In [ ]:
#@title Execute this block to start importing the libraries (**llama_index**) and helper functions

!pip install -q llama-index
!pip install -q llama-index-embeddings-huggingface
!pip install -q peft
!pip install -q auto-gptq
!pip install -q optimum
!pip install -q bitsandbytes

from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings, SimpleDirectoryReader, VectorStoreIndex
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

# Wrap the output text
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

# Download the documents
!mkdir data
!wget https://www.budget.gov.hk/2024/eng/pdf/e_budget_speech_2024-25.pdf
!mv e_budget_speech_2024-25.pdf data


In [ ]:
#@title Import an embedding model from HuggingFace hub (https://huggingface.co/spaces/mteb/leaderboard)

embeddingModel = "BAAI/bge-small-en-v1.5" # @param ["BAAI/bge-small-en-v1.5", "thenlper/gte-large"] {allow-input: true}

Settings.embed_model = HuggingFaceEmbedding(model_name=embeddingModel)

Settings.llm = None
Settings.chunk_size = 256
Settings.chunk_overlap = 25

In [ ]:
#@title Read and Store Docs into Vector DB

# articles available here: {add GitHub repo}
documents = SimpleDirectoryReader("data").load_data()

# store docs into vector DB
index = VectorStoreIndex.from_documents(documents)

In [ ]:
#@title Set Up Search Function

# set number of docs to retreive
top_k = 3

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=top_k,
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

In [ ]:
#@title Look up relevant content from the embedding

query = "What's the change in property tax in the recent HK Finanical Budget?" # @param {type:"string"}

# query documents
response = query_engine.query(query)

# reformat response
context = "Context:\n"
for i in range(top_k):
    context = context + response.source_nodes[i].text + "\n\n"

print(context)

### Import LLM

In [ ]:
# load fine-tuned model from hub
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

config = PeftConfig.from_pretrained("shawhin/shawgpt-ft")
model = PeftModel.from_pretrained(model, "shawhin/shawgpt-ft")

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

### Use LLM

In [ ]:
# prompt (no context)
intstructions_string = f"""ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and ends responses with its signature '–ShawGPT'. \
ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

Please respond to the following comment.
"""
prompt_template = lambda comment: f'''[INST] {intstructions_string} \n{comment} \n[/INST]'''

In [ ]:
comment = "What is fat-tailedness?"

prompt = prompt_template(comment)
print(prompt)

In [ ]:
model.eval()

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

In [ ]:
# prompt (with context)
prompt_template_w_context = lambda context, comment: f"""[INST]ShawGPT, functioning as a virtual data science consultant on YouTube, communicates in clear, accessible language, escalating to technical depth upon request. \
It reacts to feedback aptly and ends responses with its signature '–ShawGPT'. \
ShawGPT will tailor the length of its responses to match the viewer's comment, providing concise acknowledgments to brief expressions of gratitude or feedback, \
thus keeping the interaction natural and engaging.

{context}
Please respond to the following comment. Use the context above if it is helpful.

{comment}
[/INST]
"""

In [ ]:
prompt = prompt_template_w_context(context, comment)

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])